In [6]:
import os
import subprocess
import math
from PIL import Image

# Set file paths and directories
input_dir = "../video"
output_base_dir = "../keyframe_information/keyframe/"

# Function to reduce frame size and save as .webp
def reduce_frame(folder, new_folder, reduce_ratio=0.5):
    for frame_path in os.listdir(folder):
        full_frame_path = os.path.join(folder, frame_path)

        if os.path.isfile(full_frame_path) and frame_path.endswith('.jpg'):
            frame_name = frame_path.replace('.jpg', '.webp')
            image = Image.open(full_frame_path)

            # Get the original size and calculate the new size
            x, y = image.size
            x_new, y_new = math.floor(x * reduce_ratio), math.floor(y * reduce_ratio)

            # Resize and save the image in .webp format
            image = image.resize((x_new, y_new))
            image.save(os.path.join(new_folder, frame_name), optimize=True, quality=30, format="webp")
            print(f"Saved reduced frame: {new_folder}/{frame_name}")

# Loop through each video in the input directory
for idx, input_video in enumerate(os.listdir(input_dir), start=0):
    if input_video == ".DS_Store":
        continue
    
    input_video_path = os.path.join(input_dir, input_video)
    video_name = os.path.splitext(input_video)[0]
    
    # Create parent folder (Video1, Video2, etc.)
    parent_folder_name = f"Video{idx}"
    parent_folder_path = os.path.join(output_base_dir, parent_folder_name)
    
    # Create subdirectories for keyframes and reduced keyframes
    keyframe_dir = os.path.join(parent_folder_path, video_name)
    reduced_keyframe_dir = os.path.join(parent_folder_path, f"{video_name}_reduced")
    
    if not os.path.exists(keyframe_dir):
        os.makedirs(keyframe_dir)
    if not os.path.exists(reduced_keyframe_dir):
        os.makedirs(reduced_keyframe_dir)

    # Get the keyframe timestamps using ffprobe
    ffprobe_command = [
        "/Users/huyenphung/Downloads/ffprobe", "-loglevel", "error", 
        "-select_streams", "v:0", "-show_entries", "packet=pts_time,flags", 
        "-of", "csv=print_section=0", input_video_path
    ]

    # Run the ffprobe command and capture the output
    ffprobe_result = subprocess.run(ffprobe_command, stdout=subprocess.PIPE, text=True)

    # Filter the keyframes (lines containing ",K")
    keyframe_lines = [line.split(",")[0] for line in ffprobe_result.stdout.splitlines() if ",K" in line]

    # Extract keyframes using ffmpeg
    for index, timestamp in enumerate(keyframe_lines, start=1):
        output_image = os.path.join(keyframe_dir, f"keyframe_{index}_{timestamp}.jpg")
        
        ffmpeg_command = [
            "ffmpeg", "-loglevel", "error", 
            "-ss", timestamp, "-i", input_video_path, 
            "-frames:v", "1", output_image
        ]
        
        subprocess.run(ffmpeg_command)
        print(f"Keyframe extracted: {output_image}")

    # Reduce the extracted keyframes and save to reduced directory
    reduce_frame(keyframe_dir, reduced_keyframe_dir)

    print(f"Keyframes and reduced keyframes saved for {input_video} in {parent_folder_name}")


Keyframe extracted: ../keyframe_information/keyframe/Video0/L00/keyframe_1_0.000000.jpg
Keyframe extracted: ../keyframe_information/keyframe/Video0/L00/keyframe_2_2.502500.jpg
Keyframe extracted: ../keyframe_information/keyframe/Video0/L00/keyframe_3_3.603600.jpg
Keyframe extracted: ../keyframe_information/keyframe/Video0/L00/keyframe_4_4.804800.jpg
Keyframe extracted: ../keyframe_information/keyframe/Video0/L00/keyframe_5_5.739067.jpg
Keyframe extracted: ../keyframe_information/keyframe/Video0/L00/keyframe_6_11.244567.jpg
Keyframe extracted: ../keyframe_information/keyframe/Video0/L00/keyframe_7_19.586233.jpg
Keyframe extracted: ../keyframe_information/keyframe/Video0/L00/keyframe_8_27.927900.jpg
Keyframe extracted: ../keyframe_information/keyframe/Video0/L00/keyframe_9_36.269567.jpg
Keyframe extracted: ../keyframe_information/keyframe/Video0/L00/keyframe_10_44.611233.jpg
Keyframe extracted: ../keyframe_information/keyframe/Video0/L00/keyframe_11_52.952900.jpg
Keyframe extracted: ../k